<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.951 - Tipologia i cicle de vida de les dades</p>
<p style="margin: 0; text-align:right;">2017-2 · Màster universitari en Ciència de dades (Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;"><b>Autors:</b> Joan Bonnín i Jose L. Dolz</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>


# Pràctica 2: Neteja i validació de les dades

## Descripció 
L’objectiu d’aquesta activitat serà el tractament d’un dataset, que pot ser el creat a la pràctica 1 o bé qualsevol dataset lliure disponible a Kaggle (https://www.kaggle.com). Alguns exemples de dataset amb els que podeu treballar són:
<ul>
    <li>Red Wine Quality (https://www.kaggle.com/uciml/red-wine-quality-cortez-et-al-2009).</li>
    <li>Titanic: Machine Learning from Disaster (https://www.kaggle.com/c/titanic).</li>
    <li>Predict Future Sales (https://www.kaggle.com/c/competitive-data-science-predict-future-sales/).</li>
</ul>
Els últims dos exemples corresponen a competicions actives a Kaggle de manera que, opcionalment, podrieu aprofitar el treball realitzat durant la pràctica per entrar en alguna d’aquestes competicions.
Seguint les principals etapes d’un projecte analític, les diferents tasques a realitzar (i <b>justificar</b>) són les següents:


In [ ]:
#NOMÉS USATS PER DEV. ELIMINAR ABANS D'ENTREGAR
from pprint import pprint

# ATENCIÓ: Es important tenir instal·lats al sistema els següents
# paquets: pandas, numpy, scipy, matplotlib, sklearn, pydotplus, graphviz
#
# Si se està utilitzan Python 3, la forma d'instal·lar un paquet és
# executant la següent ordre des de la línia de comandes:
# > pyp3 install nom_paquet

# Basic maths & data structures
import pandas as pd
import numpy as np
import scipy
import scipy.stats as stats

# Data rendering
from IPython.display import display
from IPython.display import Image  
import matplotlib.pyplot as plt

# Random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

# Render tree
from sklearn.externals.six import StringIO  
from sklearn.tree import export_graphviz
import pydotplus
import graphviz

In [ ]:
# General constants
PVAL_THRESHOLD = 0.05
RANDOM_SEED = 2018
np.random.seed(RANDOM_SEED)

### 1. Descripció del dataset. Perquè és important i quina pregunta/problema pretèn respondre?

<font color="#1111aa">
    Per aquesta pràctica hem escollit el conjunt de dades de <a href="https://www.kaggle.com/uciml/red-wine-quality-cortez-et-al-2009" target="_blank">la qualitat del vi negre</a> que, com bé s'explica en la seva pàgina de Kaggle, tracta de les variants del vi portuguès conegut com <a href="http://www.vinhoverde.pt/en/" target="_blank">'Vinho Verde'</a>. Els <b>camps</b> que composen el dataset són tots <b>numérics</b> i són els següents:
    <ul>
        <li><b>fixed acidity</b>: acidesa fixa (<i>g/l</i>). Majoria dels àcids fixes o no volàtils fàcilment.</li>
        <li><b>volatile acidity</b>: acidesa volàtil (<i>g/l</i>). Quantitat d'àcid acètic que en gran quantitat porta al vi a tenir gust a vinagre.</li>
        <li><b>citric acid</b>: àcid cítric (<i>g/l</i>). Quantitat d'aquest àcid (normalment petita) que pot donar sabor i frescura als vins</li>
        <li><b>residual sugar</b>: sucre residual (<i>g/l</i>). Quantitat de sucre que queda després de la fermentació. És estrany trobar vins amb menys d'1 g/l i els que tenem més de 45 g/l es consideren dolços.</li>
        <li><b>chlorides</b>: clorurs (<i>g/l</i>). Representa la quantitat de sal al vi.</li>
        <li><b>free sulfur dioxide</b>: diòxid de sofre lliure o SO2 (<i>mg/l</i>). Parts per milió (ppm) del diòxid que queda lliure un cop es barreja en barrejar-se amb el vi. </li>
        <li><b>total sulfur dioxide</b>: Diòxid de sofre total (<i>mg/l</i>). La suma (en ppm) de la part lliure i la part fixada al vi de SO2.</li>
        <li><b>density</b>: densitat (<i>g/l</i>). Aquest valor dependrà de la quantitat d'alcohol i sucre en el vi. </li>
        <li><b>pH</b>: descriu com d'àcid o bàsic és un vi en una escala de 0 (molt àcid) a 14 (molt bàsic); La majoria dels vins són entre 3 i 4 en l'escala de pH.</li>
        <li><b>sulphates</b>: sulfats (<i>g/l</i>), que contribueixen al SO2.</li> Aquest component prevé al vi de bactèries i de l'oxidació.
        <li><b>alcohol</b>: percentatge de contigut alcohòlic en el vi.</li>
        <li><b>quality</b>: qualitat del vi en una puntuació entre el 0 i el 10.</li>
    </ul>
<p>Com podem veure, tenim els 12 atributs: 11 mesures físico-químiques i la qualitat, que podríem dir que és la classe.</p>
    <p>Aquest conjunt de dades és important perquè ens pot servir perquè, mitjançant proves de correlació, podem veure quins atributs són més influents a l'hora de millorar o empitjorar la qualitat d'un vi. A més, entrenarem un model complexe d'aprenentatge computacional que podrà predir la qualitat d'un vi, mitjançant aquestes dades físico-químiques obtingudes pels diferents instruments de medició.</p>

### 2. Integració i selecció de les dades d’interès a analitzar.

In [ ]:
wine_df = pd.read_csv('winequality-red.csv')
print("El conjunt de dades presenta {} camps i està compost per {} registres.".format(wine_df.shape[1], wine_df.shape[0]))
display(wine_df.head())

<font color="#1111aa">
    <p>Com podem observar en la taula, tots els atriuts presenten valors numérics continus que ens poden servir per cercar una relació lineal amb el valor discret de la qualitat. En aquest punt, no podem prescindir de cap d'aquests atributs doncs no sabem quina es la seva relació amb la nota de qualitat. Per tant, no ens desfarem de cap.
    </p>
    <p>
        Per altra banda, crearem un atribut categòric booleà que etiqueti si un vi és bo o dolent, depenent de la seva nota. En el nostre cas, escollim que els bon vins són aquells que tenen una puntuació de qualitat de 6 o superior.
    </p>
    </font>
        

In [ ]:
# Inservió d'atribut qualitatiu per la qualitat del vi
GOOD_THRESHOLD = 6
wine_df['is_good'] = wine_df['quality'] >= GOOD_THRESHOLD

### 3. Neteja de les dades.

#### 3.1. Les dades contenen zeros o elements buits? Com gestionaries aquests casos?

In [ ]:
# Contem els valors nuls
n_nulls = wine_df[wine_df.isnull()].count()

# Contem els zeros
n_zeros = wine_df[wine_df == 0].count()

df_empties = pd.DataFrame()
df_empties["Nombre de Nulls"] = n_nulls
df_empties["Nombre de zeros"] = n_zeros
display(df_empties.T)

<font color="#1111aa">
<p>
    Com podem observar, no tenim cap atribut que presenti valors nuls. Per altra banda, només trobem zeros en l'atribut d'àcid cítric. Concretament, tenim 132 zeros d'un total de 1599 registres, el que representa un 8'26% del total.</p>

<p>
    En aquest cas, no cal que substituim els zeros per cap altra valor. És totalment normal trobar vins negres sense àcid cítric, fet que els hi dona un sabor més anyenc. De fet, com veurem més endavant, el rang de grams per litre de l'àcid cítric en vins negres és molt petit i el zero estaria dintre dels barems. Fins i tot podem trobar vins blancs amb zero gram d'àcid cítric, tot i que en aquests acostuma a haver unes quantitats més altes que als vins negres.</p>
<p>
    Per últim, els 744 zeros que trobem a l'atribut categòric <i>is_good</i> representen els valors booleans fals. Per tant, no farem cap tractament.
    </p>
</font>

#### 3.2. Identificació i tractament de valors extrems.

In [ ]:
# Dibueixem un boxplot per cada atribut per veure màxims, 
# mínims, quartils, rang interquartilic i outliers.

df_cols = list(wine_df)
n_rows = 2
n_cols = 6

fig, axes = plt.subplots(n_rows, n_cols, figsize=(30, 15))
for i, row_axes in enumerate(axes):
    for j, ax in enumerate(row_axes):
        idx = i*n_cols + j
        wine_df.boxplot(column=df_cols[idx], ax=axes[i][j], grid=False)
        
plt.show()

<font color="#1111aa">
<p>Als boxplots s'observa que la majoria de variables presenten una quantitat elevada d'*outliers*, així que val la pena aplicar-hi una anàlisi més extensa per detectar si es tracta de presència de valors erronis o la ralitat de les dades és la que es mostra.
    </p>
    <p>
El fet que la majoria d'*outliers* s'agrupin a prop dels bigotis i la densistat baixi segons s'allunyen, ens fa pensar que els valors poden ser correctes, i que simplement la distribució presenta una desviació lateral o una variància molt elevada. Un exemple habitual per mostrar dades que presenten aquesta característica són els salaris.
        </p>
</font>    

### 4. Anàlisi de les dades.
#### 4.1. Selecció dels grups de dades que es volen analitzar/comparar (planificació dels anàlisis a aplicar).

In [ ]:
# Això ens serveix per res?????
wine_df.describe()

#### 4.2. Comprovació de la normalitat i homogeneïtat de la variància.

In [ ]:
####### NORMALITAT #######

# Mètodes gràfics 
def render_normality_histograms():
    n_rows = 3
    n_cols = 4
    n_bars = 30
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(30, 15))
    for i, row_axes in enumerate(axes):
        for j, ax in enumerate(row_axes):
            idx = i*n_cols + j

            attr = df_cols[idx]
            ax.set_xlabel(attr)      

            data = wine_df[attr].sort_values()
            ax.hist(data, bins=n_bars, density=True)

            norm = stats.norm.pdf(data, np.mean(data), np.std(data))
            ax.plot(data, norm) 
    plt.show()
        
def plot_qq():
    for attr_key in wine_df:
        attr = wine_df[attr_key]
        stats.probplot(attr, dist="norm", plot=plt)
        plt.show()

# Mètodes numèrics
def test_normality_anderson(render=False):
    normal_attrs = []
    non_normal_attrs = []
    for attr_key in wine_df:
        attr = wine_df[attr_key].values
        res = stats.anderson(attr, dist='norm')
        stat = res.statistic
        threshold = res.critical_values[2] #0.05 significance == PVAL_THRESHOLD
        if stat > threshold:
            normal_attrs.append(attr_key)
        else:
            non_normal_attrs.append(attr_key)
    
    if render:
        print_attrs_distributions(normal_attrs, non_normal_attrs)
        
def test_normality_shapiro(p_value_threshold=PVAL_THRESHOLD, render=False):
    normal_attrs = []
    non_normal_attrs = []
    for attr_key in wine_df:
        attr = wine_df[attr_key].values
        res = stats.shapiro(attr)
        if res[1] > p_value_threshold:
            non_normal_attrs.append(attr_key)
        else:
            normal_attrs.append(attr_key)
            
    if render:
        print_attrs_distributions(normal_attrs, non_normal_attrs)
        
        
def test_normality_dagostino(p_value_threshold=PVAL_THRESHOLD, render=False):
    normal_attrs = []
    non_normal_attrs = []
    for attr_key in wine_df:
        attr = wine_df[attr_key].values
        res = stats.normaltest(attr)
        if res.pvalue > p_value_threshold:
            non_normal_attrs.append(attr_key)
        else:
            normal_attrs.append(attr_key)
            
    if render:
        print_attrs_distributions(normal_attrs, non_normal_attrs)        
    
def print_attrs_distributions(normal, non_normal):
    print("Atributs amb distribució normal: {}".format(normal))
    print("Atributs amb distribució no normal: {}".format(non_normal))
    
render_normality_histograms()
#plot_qq()

print("########## Anderson - Darling ##########")
test_normality_anderson(render=True)
print("########################################", end='\n\n')

print("########## Shapiro - Wilk ##########")
test_normality_shapiro(render=True)
print("####################################", end='\n\n')

print("########## D'Agostino i Pearson ##########")
test_normality_dagostino(render=True)
print("##########################################", end='\n\n')


<font color="#1111aa">
    <p>
        Si observem els resultats de les proves sobre normalitat d'Anderson-Darling, de Saphiro-Wilk i el basat en els test de D'Agostino i Pearson, tots els atributs segueixen una distribució normal. ...
        ....
        ...
    </p>
    </font>

In [ ]:
####### HOMOGENEITAT VARIANCIA #######

# Levene és util si la normalitat no està asegurada
def levene_test(threshold=PVAL_THRESHOLD):
    res = stats.levene(*wine_df.as_matrix(), center='median') # Expand matrix to n parameters with '*'
    p = res.pvalue
    return p > threshold
    print(res)
    
    res = stats.levene([1,2,3],[0,1, 71])
    print(res)
    
levene_test()

#### 4.3. Aplicació de proves estadístiques per comparar els grups de dades. En funció de les dades i de l’objectiu de l’estudi, aplicar proves de contrast d’hipòtesis, correlacions, regressions, etc.

In [ ]:
# Mirem les correlacions entre els atributs i la qualitat
data = np.ndarray(shape=(len(df_cols), 2))
for i in range(len(df_cols)):
    attr = df_cols[i]
    data[i] = stats.pearsonr(wine_df[attr], wine_df['quality'])
df_pearson = pd.DataFrame(data, index=df_cols, columns=['estimació', 'p-valor'])[:11]
df_pearson['tmp_sort'] = df_pearson['estimació'].abs()
df_pearson = df_pearson.sort_values(by='tmp_sort', ascending=False).drop('tmp_sort', axis=1)

# Format output
# pprint(df_pearson['estimació'].values)
df_pearson['estimació'] = ["{0:.4f}".format(el) for el in df_pearson['estimació'].values]
df_pearson['p-valor'] = ["{0:.2e}".format(el) for el in df_pearson['p-valor'].values]
display(df_pearson.T)

<font color="#1111aa">
    <p>
        Hem ordenat els atributs dels que més impacten a l'hora de determinar la qualitat als que menys. Tal i com podem observar, la qualitat augmenta gairebé un 50% segons el volum d'**alcohol** que tenim en el vi. Per altra banda, el 40% de quantitat d'**acidesa volàtil** influeix negativament en la qualitat. És a dir, com més sàpiga a vinagre, pitjor.
    </p>
    <p>
        Seguidament tenim que el 25% de la quantitat de **sulfats** augmenten la qualitat. Sembla obvi, doncs aquest component ajuda a que el vi no s'oxidi i impideix l'aparició de bactèries. Però per altra banda, paradoxalment,el **SO2 total i el lliure** resten qualitat al vi (un 18'5%  i un 5% de la seva quantitat, respectivament). 
    </p>
    <p>
        L'**àcid cítric** suma el 22'6% del seu valor i podem deduïr que un vi que sigui més fresc afavoreix a obtenir una millor nota. Per altra banda, quan menys dens és el vi -menys líquid- pitjor la seva qualitat, restant un 17'5% del valor de la **densitat**. La quantitat de **clorurs** també afecta negativament, restant casi un 13% del seu valor: quant més salat estigui el vi, pitjor qualitat tindrà. Per altra banda, el valor del **sucre** només suma un 1'4% del seu valor. No ens ha de resultar xocant doncs estem parlant de vins negres i aquests no acostumen a ser dolços (tret que siguin escumosos, però no és el cas d'aquests). 
    </p>
    <p>
        Darrerament, tenim que el **pH**, que resta un 5% del seu valor a la qualitat final. És a dir, com més àcid sigui un vi, pijtor encara que no compti molt. Però, per altra banda, l'**acidesa fixa** suma a la qualitat del vi un 12'4% del seu valor. 
    </p>
    <p>
        Pel que fa als p-valors, podem descartar tots aquells atributs que superin el llindar de 0.05. Només el sucre supera aquest llindar i, per tant, no el fariem servir en un model de regresió lineal.
    </p>
</font>
    

In [ ]:
# Correlació dels atributs
pd.plotting.scatter_matrix(wine_df.iloc[:, :11], figsize=(30,15))
plt.show()

<font color="#1111aa">
    <p>En el gràfic de correlacions entre parells d'atributs -considerant que són tots normals- podem veure clarament la relació directe que havíem explicat al principi entre el SO2 lliure i total: com més tenim d'un, més tenim de l'altre.</p>
     <p>
        Per altra banda, tenim atributs que es relacionen fortament entre sí. Per exemple, com més àcid cítric, més àcidesa fixa. Això ens fa pensar que pujant aquest dos paràmetres obtíndriem millor nota perquè, a més, com més àcidesa fixa, menys pH, que li resta qualitat. A més a més, com més àcid cítric, menys acidesa volàtil i, per tant, menys sabor a vinagre. Però tenim el problema que com més quantitat d'acidesa fixa, derivada de l'àcid cítric, augmentem també la seva densitat, que va en detriment de la qualitat. Com veiem, trobar els paràmetres físico-químics equilibrats per treure un vi de molt bona qualitat és una tasca força feixuga. 
    </p>
    <p>
        A continuació, construïrem un model de predicció. Será una combinació d'arbres de decisió entrenats amb *bagging* (usant mostreig amb reemplaçament), també conegut com **Random Forests**.
    </p>
</font>

In [ ]:
def random_forest_preprocess_data(df):
    #Input data
    X = df.drop(['quality', 'is_good'], axis=1)
    #Labels
    le = preprocessing.LabelEncoder()
    y = le.fit_transform(df['is_good'])
    return (X, y)

def split_train_test(X, y):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.15, random_state=RANDOM_SEED
    )
    return (X_train, X_test, y_train, y_test)

(X, y) = random_forest_preprocess_data(wine_df)
(X_train, X_test, y_train, y_test) = split_train_test(X, y)

rfc_model = RandomForestClassifier(bootstrap=True, n_estimators = 150, n_jobs=8, random_state=RANDOM_SEED)
rfc_model.fit(X_train, y_train)

In [ ]:
rfc_score = rfc_model.score(X_test, y_test)
print("Precisió del model: {} %".format(round(rfc_score*100, 2)))
# pprint(rfc_model.feature_importances_)
importances_df = pd.DataFrame(
    rfc_model.feature_importances_,
    index=list(X_test),
    columns=['Importància']
)
importances_df = importances_df.sort_values(by="Importància", ascending=False)
display(importances_df.T)

<font color="#1111aa">
    <p>Com podem veure, els pesos que otorga als atributs el nostre model basat en boscos aleatoris són bastant semblants als obtinguts en la correlació lineal, sent l'alcohol el de major pes i el sucre residual el que menys.
    </p>
    <h2> Expliquem perquè són tots positius?? </h2>
    <p>
        Per altra banda, obtenim una precisió prou alta encara que una mica per sota del llindar del 80% al que estem acostumats a donar per bo el model. Hem de tenir en compte que un model complexe com són els *Random Forests* millora la seva precisió com més gran és el seu conjunt d'entrenament. Per tant, podem asegurar que utilizant les dades dels vins de collites d'altres anys -recordem que estem fent servir només l'any 2009- millorarem la precisió de manera que aquest model podrà determinar de forma més exacte la qualitat de vins futurs.
    </p>
    </font>

### 5. Representació dels resultats a partir de taules i gràfiques.

In [ ]:
# Render random forest sample tree
sample_tree = np.random.choice(rfc_model.estimators_)

dot_data = StringIO()
export_graphviz(sample_tree, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True)

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

### 6. Resolució del problema. A partir dels resultats obtinguts, quines són les conclusions? Els resultats permeten respondre al problema?

<font color="#1111aa">
    <p>
        Com hem pogut veure al llarg de tot el document, tots els atributs presents són necesaris per determinar la qualitat del vi. No tots tenen la mateixa importància a l'hora de la seva classificació, però fins i tot el 'menys important' pot ser clau per diferenciar un vi de bona qualitat d'un que no.
    </p>
    <p>
        Per altra banda, hem construit un model bastant robust que pot classificar els vins entre bons i, diguem-ne, no tant bons, amb una fiabilitat de gairebé el 79%. Com ja s'ha comentat, si a aquest model li donem dades d'anys posteriors a l'any 2009 per entrenar-se, obtíndrem un classificador molt fiable per determinar la qualitat del vins d'enguany.
    </p>
    </font>

### 7. Codi: Cal adjuntar el codi, preferiblement en R, amb el que s’ha realitzat la neteja, anàlisi i representació de les dades. Si ho preferiu, també podeu treballar en Python

Tot el codi emprat en aquesta pràctica es pot trobar en les diferens preguntes en les cel·les precedides amb `In [ ]` i es poden executar des de jupyter.